In [ ]:
# Add table of contents

# Libraries and Notebook Options

In [1]:
import pandas as pd
import numpy as np
import pickle

# # Exploratory Data Analysis
# from pandas_profiling import ProfileReport

# SODA API

In [2]:
# from sodapy import Socrata
# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# client = Socrata('data.sfgov.org', None)

# # Authenticate the client
# domain = 'data.sfgov.org'
# app_token = 'tZuHs7ko1xQezRBAgUN10D0i9'
# username = 'corraljrmiguel@gmail.com'
# password = 'Xd9@rzwPhZjKsNg8cima'

# client = Socrata(domain,
#                  app_token,
#                  username=username,
#                  password=password)

# # Returns as JSON from API
# # converted to Python list of dictionaries by sodapy.
# results = client.get('vw6y-z8j6', limit=10000)
# # results = client.get('vw6y-z8j6', where='service_request_id=12167455')

# # Convert to pandas dataframe
# df = pd.DataFrame.from_records(results)

# Data Cleaning

In [3]:
# # Import CSV – 4.1M rows
# df = pd.read_csv('../data/311_Cases.csv')
df = pd.read_csv('../data/rows.csv')

# # [TEMPORARY]
# df = read_pickle('../data/df_raw_all.pkl')
# df = df.sample(n=20000, random_state=2020)

# Sort dataframe by CaseID (newest first)
df.sort_values(by='CaseID', ascending=False, inplace=True)

# # Set index to CaseID
# df.set_index('CaseID', drop=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

df.shape

(4107366, 47)

In [4]:
# Drop all computed columns (unnecessary)
df = df.iloc[:, 0:19]
# df = df.loc[:, ~df.columns.str.contains('^:@')]

print(df.shape)
df.head()

(4107366, 19)


,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source
0,12214650,03/14/2020 09:35:00 AM,03/14/2020 09:39:41 AM,03/14/2020 09:39:41 AM,Closed,Case is a Duplicate - This issue has already b...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Tow,Black - BMW/x2 - R540y0,"545 ASHBURY ST, SAN FRANCISCO, CA, 94117",ASHBURY ST,5.0,Haight Ashbury,PARK,37.770263,-122.447232,"(37.7702626, -122.4472325)",Web
1,12214645,03/14/2020 09:34:00 AM,03/14/2020 11:22:00 AM,03/14/2020 11:22:00 AM,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Tow,Grey - Ford//focus - 7aph120,"545 ASHBURY ST, SAN FRANCISCO, CA, 94117",ASHBURY ST,5.0,Haight Ashbury,PARK,37.770263,-122.447232,"(37.7702626, -122.4472325)",Web
2,12214640,03/14/2020 09:33:02 AM,03/14/2020 11:30:59 AM,03/14/2020 11:30:59 AM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Refrigerator,"582 48TH AVE, SAN FRANCISCO, CA, 94121",48TH AVE,1.0,Sutro Heights,RICHMOND,37.777370,-122.509247,"(37.77737, -122.50924667)",Mobile/Open311
3,12214639,03/14/2020 09:32:55 AM,03/14/2020 10:12:17 AM,03/14/2020 10:12:17 AM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,"11 AMAZON AVE, SAN FRANCISCO, CA, 94112",AMAZON AVE,11.0,Excelsior,INGLESIDE,37.717062,-122.439998,"(37.71706177, -122.4399978)",Mobile/Open311
4,12214638,03/14/2020 09:32:00 AM,03/14/2020 09:48:30 AM,03/14/2020 09:48:30 AM,Closed,Case Resolved - See CAD# 200740897 for details.,Parking Enforcement Dispatch Queue,Parking Enforcement,Other_Illegal_Parking,Silver - Nissan Altima - 6JWK906,"2136 FUNSTON AVE, SAN FRANCISCO, CA, 94116",FUNSTON AVE,7.0,Golden Gate Heights,TARAVAL,37.748255,-122.469175,"(37.748255, -122.469175)",Mobile/Open311


In [5]:
# Strip column names and format
df.columns = df.columns.str.strip().str.lower().str.replace(
    ' ', '_').str.replace('(', '').str.replace(')', '')

# Rename 'caseid' column
df.rename({'caseid': 'case_id'}, axis='columns', inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4107366 entries, 0 to 4107365
Data columns (total 19 columns):
case_id                int64
opened                 object
closed                 object
updated                object
status                 object
status_notes           object
responsible_agency     object
category               object
request_type           object
request_details        object
address                object
street                 object
supervisor_district    float64
neighborhood           object
police_district        object
latitude               float64
longitude              float64
point                  object
source                 object
dtypes: float64(3), int64(1), object(15)
memory usage: 595.4+ MB


In [6]:
# Convert time columns to datetime
df['opened'] = pd.to_datetime(df['opened'])
df['closed'] = pd.to_datetime(df['closed'])
df['updated'] = pd.to_datetime(df['updated'])
# df.iloc[:, 1:4] = pd.to_datetime(df.iloc[:, 1:4].stack()).unstack()

df.shape

(4107366, 19)

In [7]:
# # Remove incomplete years (2008 & 2020)
# df = df.loc[(df['opened'] >= '2009-01-01') & (df['opened'] <= '2019-12-31')]

# TEMP
df = df.loc[(df['opened'] >= '2019-01-01') & (df['closed'] <= '2019-01-31')].sample(5000)

# Drop all rows with any missing value
df.dropna(how='any', inplace=True)

# Drop 'status' column as all incidents are 'closed' after running the code above
df.drop('status', axis=1, inplace=True)

df.shape

(4846, 18)

In [8]:
# Convert categorical columns to 'category' type
df['supervisor_district'] = pd.to_numeric(df['supervisor_district'], downcast='integer').astype('category')

cols = df[['category', 'neighborhood', 'police_district', 'source']]
for col in cols:
    df[col] = df[col].astype('category')
    
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4846 entries, 772129 to 780093
Data columns (total 18 columns):
case_id                4846 non-null int64
opened                 4846 non-null datetime64[ns]
closed                 4846 non-null datetime64[ns]
updated                4846 non-null datetime64[ns]
status_notes           4846 non-null object
responsible_agency     4846 non-null object
category               4846 non-null category
request_type           4846 non-null object
request_details        4846 non-null object
address                4846 non-null object
street                 4846 non-null object
supervisor_district    4846 non-null category
neighborhood           4846 non-null category
police_district        4846 non-null category
latitude               4846 non-null float64
longitude              4846 non-null float64
point                  4846 non-null object
source                 4846 non-null category
dtypes: category(5), datetime64[ns](3), float64(2), int64(1

In [9]:
# Create 'is_duplicate' column
def create_is_duplicate(df):
    """Create 'is_duplicate' column"""
    
    # Search for duplicates in 'status_notes'
    df.insert(loc=1, column='is_duplicate', 
              value=df['status_notes'].str.lower().str.contains('dupe|duplicate', regex=True).astype('int'))
    
    # Fill 'nan' values with False
    df['is_duplicate'].fillna(False, inplace=True)
    
    return df

create_is_duplicate(df)

# Move target variable to 2nd index


df.head()

,case_id,is_duplicate,opened,closed,updated,status_notes,responsible_agency,category,request_type,request_details,address,street,supervisor_district,neighborhood,police_district,latitude,longitude,point,source
772129,10390775,0,2019-01-22 14:02:41,2019-01-22 18:08:07,2019-01-22 18:08:07,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Mattress,"290 TRUMBULL ST, SAN FRANCISCO, CA, 94112",TRUMBULL ST,11,Mission Terrace,INGLESIDE,37.730783,-122.424126,"(37.730783, -122.424126)",Mobile/Open311
790082,10347011,0,2019-01-11 20:08:06,2019-01-12 12:47:43,2019-01-12 12:47:43,Case Resolved - WASTE NOT FOUND,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Electronics,"319 SILVER AVE, SAN FRANCISCO, CA, 94112",SILVER AVE,11,Mission Terrace,INGLESIDE,37.728667,-122.430557,"(37.72866734, -122.4305567)",Mobile/Open311
788739,10349923,0,2019-01-12 20:36:00,2019-01-14 07:16:09,2019-01-14 07:16:09,Encampment Removed\nhandled,HSOC Queue,Encampments,Encampment Reports,Encampment Cleanup,Intersection of CHURCH ST and 18TH ST,CHURCH ST,8,Mission Dolores,MISSION,37.761288,-122.428352,"(37.76128769, -122.42835236)",Phone
773049,10388187,1,2019-01-22 09:08:00,2019-01-22 09:50:55,2019-01-22 09:50:55,Case is a Duplicate - This issue has already b...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Only,Blue - BMW - 7vfv578,"2204 STEINER ST, SAN FRANCISCO, CA, 94115",STEINER ST,2,Pacific Heights,NORTHERN,37.789950,-122.435616,"(37.78994958, -122.43561595)",Mobile/Open311
782771,10364426,0,2019-01-16 12:26:35,2019-01-16 15:44:07,2019-01-16 15:44:07,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Furniture,"2410 VAN NESS AVE, SAN FRANCISCO, CA, 94109",VAN NESS AVE,3,Polk Gulch,NORTHERN,37.797686,-122.423524,"(37.79768623, -122.42352412)",Mobile/Open311


In [10]:
# profile = ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
# profile.to_notebook_iframe()

In [ ]:
# # Pickle dataframe
# df.to_pickle('../data/df_clean_20k.pkl')

In [11]:
# TEMP
df.to_pickle('../data/df_clean_5k_Jan19.pkl')